In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MP_API_KEY")
m = MPRester(api_key = api_key)

import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric
import torch_scatter

import e3nn
from e3nn import rs, o3
from e3nn.util.datatypes import DataPeriodicNeighbors
from e3nn.nn import GatedConvParityNetwork
from e3nn.o3 import Kernel
from e3nn.nn import Convolution

import pymatgen as mg
import pymatgen.io
from pymatgen.core.structure import Structure
from pymatgen.ext.matproj import MPRester
import pymatgen.analysis.magnetism.analyzer as pg
import numpy as np
import pickle
from mendeleev import element
import matplotlib.pyplot as plt

from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import io
import random
import math
import sys 
import time, os
import datetime


#%% Process Materials Project Data
order_list_mp =  []
structures_list_mp = []
formula_list_mp = []
sites_list = []
id_list_mp = []
y_values_mp = []
order_encode = {"NM": 0, "AFM": 1, "FM": 2, "FiM": 2}


m = MPRester(api_key= m, endpoint=None, include_user_agent=True) 
structures = m.query(criteria={"elements": {"$in":['Ga', 'Tm', 'Y', 'Dy', 'Nb', 'Pu', 'Th', 'Er', 'U', 'Cr', 'Sc', 'Pr', 'Re', 'Ni', 'Np', 'Nd', 'Yb', 'Ce', 'Ti', 'Mo', 'Cu', 'Fe', 'Sm', 'Gd', 'V', 'Co', 'Eu', 'Ho', 'Mn', 'Os', 'Tb', 'Ir', 'Pt', 'Rh', 'Ru']}, 'blessed_tasks.GGA+U Static': {'$exists': True}}, properties=["material_id","pretty_formula","structure","blessed_tasks", "nsites"])


In [ ]:
import torch
from datetime import datetime

In [ ]:
time_stamp = datetime.today().strftime('%Y-%m-%d_%H-%M')

In [ ]:
struct_dict = {
    "structures": structures
}

In [ ]:
torch.save(struct_dict, f'mp_structures_{time_stamp}.pt')

In [ ]:
structures_loaded = torch.load(f'mp_structures_{time_stamp}.pt')